In [1]:
import pandas as pd

from helper import *
from nse_func import *

# Do assignments
a = assign_var('nse')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()
ib = get_connected('nse', 'live')

In [2]:
# Code to set up harvest quotes
# ______________________________
p = util.df(ib.portfolio()) # portfolio table

# extract option contract info from portfolio table
dfp = pd.concat([p, util.df([c for c in p.contract])[util.df([c for c in p.contract]).columns[:7]]], axis=1).iloc[:, 1:]
dfp = dfp.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

dfp = dfp.assign(dte=[get_dte(d) for d in dfp.expiration])

# get the latest prices
contracts = ib.qualifyContracts(*[Contract(conId=c) for c in dfp.conId])

tickers = ib.reqTickers(*contracts)
optPrices = {t.contract.conId: t.marketPrice() for t in tickers} # {conId: optPrice}

# make the expPrice to be the price minimum of df.marketPrice*0.9 and hvstPrice()
dfp1 = dfp.assign(hvstPrice=[get_prec(min(hvstPricePct(d),mp*0.9),0.05) for d, mp in zip(dfp.dte, dfp.marketPrice)])
dfp1 = dfp1.loc[:, ['position', 'marketPrice', 'averageCost', 'secType', 'conId', 'symbol', 'expiration', 'strike', 'right', 'dte', 'hvstPrice']]

dfp1 = dfp1.assign(order=[LimitOrder(action='BUY', 
                                     totalQuantity=-position, lmtPrice=hvstPrice) 
                          for position, hvstPrice in zip(dfp1.position, dfp1.hvstPrice)])

In [3]:
dfp1

,position,marketPrice,averageCost,secType,conId,symbol,expiration,strike,right,dte,hvstPrice,order
0,-1500.0,0.558501,2.234169,OPT,355570106,JSWSTEEL,20190530,235.0,P,31,0.25,"LimitOrder(action='BUY', totalQuantity=1500.0,..."
1,-1500.0,0.705961,2.933392,OPT,355570114,JSWSTEEL,20190530,240.0,P,31,0.25,"LimitOrder(action='BUY', totalQuantity=1500.0,..."


In [19]:
df_hvstTrades = pd.concat([pd.DataFrame(harvestTrades), util.df(harvestTrades)], axis=1)
df_hvstTrades.columns=[['trade']+list(df_hvstTrades)[1:]]
df_hvstTrades.to_pickle(fspath+'_hvstTradesDone.pickle')

In [ ]:
df_hvstTrades

In [ ]:
# Code to set up sowing orders
# ____________________________
dft = pd.read_pickle(fspath+'_targetopts.pickle')
grp_opts(dft)

In [ ]:
df_trades = pd.concat([pd.DataFrame(trades), util.df(trades)], axis=1)
df_trades.columns=[['trade']+list(df_trades)[1:]]
df_trades.to_pickle(fspath+'_tradesDone.pickle')

In [ ]:
df_trades

In [ ]:
# Execute the trades
def xcute(ib, fspath):
    '''Executes trades
    Args:
       (ib) as connection object
       (fspath) as string with path to pickles
    Returns:
       None. But pickles _tradesdone.pickle'''
    
    # read from the targets
    dft = pd.read_pickle(fspath+'_targetopts.pickle')